In [1]:
import json
import pandas as pd
import warnings
import tqdm
import os
import string
import re
import csv
warnings.filterwarnings('ignore')
from rouge import Rouge
rouge = Rouge()
import evaluate

2023-11-07 23:08:28.411832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 23:08:28.579046: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 23:08:30.119332: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-07 23:08:30.119526: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
bertscore = evaluate.load('bertscore')
em = evaluate.load('exact_match')

In [3]:
preds = []
refs = []

In [4]:
from collections import defaultdict

In [5]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [6]:
langScoreRefs = {'hi':[],'en':[],'bn':[],'ta':[],'or':[],'pa':[]}
langScorePreds = {'hi':[],'en':[],'bn':[],'ta':[],'or':[],'pa':[]}
langScore = {'hi':[],'en':[],'bn':[],'ta':[],'or':[],'pa':[]}
fin = open('final_article.csv', mode ='r')
csvData = csv.reader(fin)
for i,lineData in enumerate(csvData):
    if i == 0:
        continue
    lang = lineData[1][:2]
    x = lineData[3].split(' ')
    y = lineData[2].split(' ')
    
    x = [i for i in x if i!= '']
    y = [i for i in y if i!= '']
    
    if len(x) > len(y):
        y = y + ['<pad>']*(len(x) - len(y))
    elif len(y) > len(x):
        x = x + ['<pad>']*(len(y) - len(x))
        
#     results = em.compute(predictions=x, references=y)
#     langScore[lang].append(round(results['exact_match'], 2))
#     results = bertscore.compute(predictions=[lineData[3]], references=[lineData[2]], model_type="bert-base-multilingual-cased")
#     langScore[lang].append(results['f1'][0])
    
    langScorePreds[lang].extend(x)
    langScoreRefs[lang].extend(y)
fin.close()

In [8]:
for key in langScore.keys():
    if len(langScore[key]) != 0:
        print(f"For language: {key}, average Exact Match score is: {sum(langScore[key])/len(langScore[key])}")

In [9]:
scores = defaultdict(float)
for ln in langScoreRefs.keys():
    y_true = langScoreRefs[ln]
    y_pred = langScorePreds[ln]
    metric_dict = {'acc': accuracy_score(y_true, y_pred),
                    'p': precision_score(y_true, y_pred,average='weighted'),
                    'r': recall_score(y_true, y_pred,average='weighted'),
                    'f1': f1_score(y_true, y_pred,average='weighted')}
    
    print(ln, metric_dict)
    
#     score = rouge.get_scores(all_preds, all_refs, avg=True)
#     scores[ln] = score['rouge-l']['f']

hi {'acc': 0.630667578035999, 'p': 0.6576921870681213, 'r': 0.630667578035999, 'f1': 0.640187223670143}
en {'acc': 0.46185286103542234, 'p': 0.5250970933780474, 'r': 0.46185286103542234, 'f1': 0.4759085933734836}
bn {'acc': 0.5898498187467633, 'p': 0.5013551158220036, 'r': 0.5898498187467633, 'f1': 0.5278847985683925}
ta {'acc': 0.6036179835062516, 'p': 0.7498223206932829, 'r': 0.6036179835062516, 'f1': 0.65691687773671}
or {'acc': 0.43284671532846714, 'p': 0.6185370812282622, 'r': 0.43284671532846714, 'f1': 0.49426936096750496}
pa {'acc': 0.3891242937853107, 'p': 0.4442298121478621, 'r': 0.3891242937853107, 'f1': 0.40775863227103276}


In [10]:
scores

defaultdict(float, {})

In [6]:
langScore = {'hi':[],'en':[],'bn':[],'ta':[],'or':[],'pa':[]}
for lang in tqdm.tqdm(langScore.keys()):
    gptfin = open(f'clfe_scrapes/merged_outputs/{lang}_gpt.csv', mode ='r')
    reffin = open(f'/scratch/shivansh.s/intersection/{lang}_test.json', mode='r')
    gptData = csv.reader(gptfin)
    refData = []
    for line in reffin:
        refData.append(json.loads(line))
    for i,lineData in enumerate(gptData):
        if i == 0:
            continue
        for reflineData in refData:
            if lineData[4] == reflineData["qid"]:
                refFact = ""
                for fact in reflineData["xalign_facts"]:
                    refFact += fact[0].lower() + " " + fact[1].lower() + " "
                    for punc in string.punctuation:
                        refFact = refFact.replace(punc,' ')
                gptFact = lineData[2]
                gptFact = re.sub(r'<\|im_end\|>',r'', gptFact.lower())
                for punc in string.punctuation:
                    gptFact = gptFact.replace(punc,' ')
                gptFact = re.sub(' +', ' ', gptFact.strip())
                refFact = re.sub(' +', ' ', refFact.strip())
                # score = rouge.get_scores(gptFact, refFact, avg=True)
                # langScore[lang].append(score['rouge-l']['f'])

                score = bertscore.compute(predictions=[gptFact], references=[refFact], model_type="bert-base-multilingual-cased")
                langScore[lang].append(score['f1'][0])    
    gptfin.close()
    reffin.close()

100%|██████████| 6/6 [00:45<00:00,  7.60s/it]


In [40]:
for key in langScore.keys():
    print(f"Rouge-L score for lang: {key} is {sum(langScore[key])/len(langScore[key])}")

Rouge-L score for lang: hi is 0.5963631718622646
Rouge-L score for lang: en is 0.6563214898029931
Rouge-L score for lang: bn is 0.9022970499499996
Rouge-L score for lang: ta is 0.7659665247600201
Rouge-L score for lang: or is 0.599688542430034
Rouge-L score for lang: pa is 0.6006571739311406


In [7]:
for key in langScore.keys():
    print(f"Bert score for lang: {key} is {sum(langScore[key])/len(langScore[key])}")

Bert score for lang: hi is 0.832969351051506
Bert score for lang: en is 0.868092382395709
Bert score for lang: bn is 0.9537793087287688
Bert score for lang: ta is 0.9018094785073224
Bert score for lang: or is 0.8222764887436917
Bert score for lang: pa is 0.84666516322356
